# Chat Bot

## ChatBot process 
1. 의도 파악: 질문의 의도(intend) 파악 - 정치, 금융, 인물, 날씨.. => 의도 파악 모델
2. 객체 탐지 : 핵심 주제, 키워드 파악 - 주식: 종목, 가격, 회사개요, 회사뉴스.. => 객체 탐지 모델
3. 답변 리턴 : 의도 + 객체 에 맞는 answer를 찾아서 리턴

## 주식 관련 챗봇
1. 텍스트에서 의도를 탐지하는 모델을 학습 생성 - 의도
2. 객체(핵심키워들)를 탐지하는 모델 - 키워드

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import FinanceDataReader as fdr

In [6]:
df_krx = fdr.StockListing('KRX')

In [7]:
df_krx.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [10]:
fdr.DataReader('005930','2021')

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170
...,...,...,...,...,...,...
2022-04-11,67800,68100,67400,67900,12263735,0.001475
2022-04-12,67600,67700,67000,67000,13924389,-0.013255
2022-04-13,67300,69000,67200,68700,17378619,0.025373


In [12]:
df_krx1 = df_krx[['Symbol','Market','Name','Sector','Industry']].copy()

In [23]:
df_krx1['Market'].value_counts()

KOSPI     5527
KOSDAQ    1569
KONEX      128
Name: Market, dtype: int64

In [26]:
df_krx1 = df_krx1[df_krx1['Market']=='KOSPI']
df_krx1.head()

,Symbol,Market,Name,Sector,Industry
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)"
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업
6,152100,KOSPI,ARIRANG 200,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN


In [30]:
# 대략적인 펀드 제거

df_krx2 = df_krx1[df_krx1['Sector'].str.contains('금융') == False]
df_krx2.head()

,Symbol,Market,Name,Sector,Industry
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)"
52,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점
55,001460,KOSPI,BYC,봉제의복 제조업,"메리야스,란제리 제조,도매/건축공사/부동산 임대,분양,공급"
59,079160,KOSPI,CJ CGV,"영화, 비디오물, 방송프로그램 제작 및 배급업","영화상영,영화관 운영"
63,000120,KOSPI,CJ대한통운,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM..."


In [31]:
# 종목이름 리스트로 뽑기

company_list = df_krx2['Name'].tolist()
company_list

['AJ네트웍스',
 'BGF리테일',
 'BYC',
 'CJ CGV',
 'CJ대한통운',
 'CJ씨푸드',
 'CJ제일제당',
 'DB',
 'DB손해보험',
 'DB하이텍',
 'DI동일',
 'DL건설',
 'DL이앤씨',
 'DRB동일',
 'DSR',
 'DSR제강',
 'E1',
 'ESR켄달스퀘어리츠',
 'F&F',
 'GKL',
 'GS건설',
 'GS글로벌',
 'GS리테일',
 'HDC랩스',
 'HDC현대EP',
 'HDC현대산업개발',
 'HD현대',
 'HJ중공업',
 'HMM',
 'HSD엔진',
 'IHQ',
 'JW생명과학',
 'JW중외제약',
 'KCC',
 'KCC글라스',
 'KCTC',
 'KC코트렐',
 'KEC',
 'KG스틸',
 'KG케미칼',
 'KH 필룩스',
 'KISCO홀딩스',
 'KPX케미칼',
 'KPX홀딩스',
 'KR모터스',
 'KSS해운',
 'KT',
 'KT&G',
 'KTcs',
 'KTis',
 'LF',
 'LG디스플레이',
 'LG생활건강',
 'LG에너지솔루션',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG헬로비전',
 'LG화학',
 'LIG넥스원',
 'LS ELECTRIC',
 'LS네트웍스',
 'LX인터내셔널',
 'LX하우시스',
 'MH에탄올',
 'NAVER',
 'NHN',
 'NH올원리츠',
 'NH프라임리츠',
 'NICE',
 'NI스틸',
 'NPC',
 'OCI',
 'PI첨단소재',
 'POSCO홀딩스',
 'S-Oil',
 'SBS',
 'SGC에너지',
 'SG글로벌',
 'SG세계물산',
 'SHD',
 'SH에너지화학',
 'SIMPAC',
 'SJM',
 'SJM홀딩스',
 'SKC',
 'SK가스',
 'SK네트웍스',
 'SK디앤디',
 'SK렌터카',
 'SK리츠',
 'SK바이오사이언스',
 'SK바이오팜',
 'SK아이이테크놀로지',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'SK하이닉스',
 'SNT

In [15]:
from konlpy.tag import Komoran
kom = Komoran()

In [16]:
kom.pos('삼성전자 시가 알려줘')

[('삼성전자', 'NNP'),
 ('시가', 'NNP'),
 ('알리', 'VV'),
 ('어', 'EC'),
 ('주', 'VX'),
 ('어', 'EC')]

In [17]:
# 하고 라는 표현 추가하여 데이터 증강 필요
names = stock_list
def question_generator(names):
    question = []
    for name in names:
        s1 = name+'의 유사 종목 알려줘'
        s2 = name+'의 유사 종목 알고싶어'
        s3 = name+'의 유사 종목 알려줄래?'
        s4 = name+'의 유사 종목은?'
        s5 = name+' 유사 종목 검색'
        s5 = name+'랑 유사한 주식 알고싶어'
        s6 = name+'와 유사한 주식이 모야'
        s7 = name+' 유사 종목 검색해줘'
        s8 = name+'의 비슷한 종목 알려줘'
        s9 = name+'와 비슷 종목 검색해줘'
        s10 = name+'와 비슷한 주식을 알려줘'
        s11 = name+'과 비슷한 종목을 알고싶어'
        s12 = '유사 종목 검색 '+name
        s13 = '유사 종목 '+name
        s14 = '비슷한 종목 검색 '+name
        s15 = '유사 종목 분석 '+name
        s16 = name+'와 비슷한 종목 분석'
        s17 = name+'과 유사 종목 분석'
        s18 = name+'랑 유사한 종목이 궁금해'
        s19 = name+'이랑 비슷한 주식을 알려줘'
        question = question+[s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19]
    return question
question = question_generator(names)

In [ ]:
def question_generator(random_names):
    question = []
    for name in random_names:
        s1 = name+' 관련주 알려줘'
        s2 = name+'랑 관련있는 주식 알려줘'
        s3 = name+' 관련주 검색해줘'
        s4 = name+'이랑 관련있는 종목 알려줘'
        s5 = name+' 키워드 검색'
        s6 = '키워드 검색 '+name
        s7 = name+' 테마주 검색'
        s8 = name+'의 테마 종목이 뭐야'
        s9 = name+'의 테마 주식 뭐야'
        s10 = '관련 종목 검색 '+name
        s11 = name+'과 관련있는 주식 검색'
        s12 = name+'와 관계있는 주식 검색'
        s13 = '테마주 검색 '+ name
        s14 = '관련주 검색 '+name
        s15 = '키워드 분석 '+name
        s16 = '관계주 분석 '+name
        s17 = name+'와 연관있는 주식 알려줘'
        s18 = name+'의 연관주 검색'
        s19 = name+'의 테마주 알려줘'
        s20 = name+'의 관련 종목이 궁금해'
        s21 = name+'과 관계 있는 종목이 궁금해'
        s22 = name+'로 키워드 검색 해줘'
        s23 = '키워드 '+name+'로 찾아봐'
        s24 = name+'이랑 관계있는 종목이 궁금해'
        s25 = name+' 관계주 검색'
        s26 = name+'하고 관련된 주식 알려줘'
        question = question+[s1, s2, s3, s4, s5, s6, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26]
    return question
keyword = question_generator(random_names)